In [2]:
from bs4 import BeautifulSoup as bs
import geopy.distance
import numpy as np
import pandas as pd
import re
import requests

In [3]:
url = 'https://en.wikipedia.org/wiki/User:Michael_J/County_table'
table_class = 'wikitable sortable jquery-tablesorter'
response = requests.get(url)
print(response)

soup = bs(response.text, 'html.parser')
table = soup.find('table',{'class':'wikitable'})

<Response [200]>


In [4]:
table_df = pd.read_html(str(table))
table_df = pd.DataFrame(table_df[0])

table_df = table_df.drop(['Sort [1]', 'County Seat(s) [3]', 'Population(2010)', 'Land Areakm²', 'Land Areami²', 'Water Areakm²', 'Water Areami²', 'Total Areakm²', 'Total Areami²'], axis=1)
table_df = table_df.set_axis(['State', 'Geoid', 'County', 'Lat', 'Long'], axis=1)
table_df.Lat = table_df.Lat.str.replace('+', '').str.replace('°', '')
table_df.Long = table_df.Long.str.replace('–', '-').str.replace('°', '')

C:\Users\Alex7\AppData\Local\Temp/ipykernel_2748/295409627.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  table_df.Lat = table_df.Lat.str.replace('+', '').str.replace('°', '')


In [5]:
table_df.County = table_df.County.str.replace(r'( \[.*?\])', '')

C:\Users\Alex7\AppData\Local\Temp/ipykernel_2748/2107063881.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  table_df.County = table_df.County.str.replace(r'( \[.*?\])', '')


In [6]:
stadiums_df = pd.read_excel('MLB Stadiums.xlsx').reset_index(drop=True)
stadiums_df

,Team,Abbreviation,Stadium,Latitude,Longitude,League,AL/NL
0,Arizona Diamondbacks,ARI,Chase Field,33.445278,-112.066944,MLB,NL
1,Atlanta Braves,ATL,Truist Park,33.890781,-84.468239,MLB,NL
2,Baltimore Orioles,BAL,Oriole Park at Camden Yards,39.283889,-76.621667,MLB,AL
3,Boston Red Sox,BOS,Fenway Park,42.346389,-71.097500,MLB,AL
4,Chicago Cubs,CHC,Wrigley Field,41.948333,-87.655556,MLB,NL
5,Chicago White Sox,CHW,Guaranteed Rate Field,41.830000,-87.633889,MLB,AL
6,Cincinnati Reds,CIN,Great American Ball Park,39.097500,-84.506667,MLB,NL
7,Cleveland Guardians,CLE,Progressive Field,41.495833,-81.685278,MLB,AL
8,Colorado Rockies,COL,Coors Field,39.756111,-104.994167,MLB,NL
9,Detroit Tigers,DET,Comerica Park,42.339167,-83.048611,MLB,AL


In [20]:
min_distance = []
for i, j in zip(table_df.Lat, table_df.Long):
    county_coords = (float(i), float(j))
    distance_list = []
    for i, j in zip(stadiums_df.Latitude, stadiums_df.Longitude):
        stadium_coords = (float(i), float(j))
        distance_list.append(geopy.distance.distance(county_coords, stadium_coords).mi)
    min_distance.append(min(distance_list))

table_df['min_distance'] = min_distance

max_distance = []
for i, j in zip(table_df.Lat, table_df.Long):
    county_coords = (float(i), float(j))
    distance_list = []
    for i, j in zip(stadiums_df.Latitude, stadiums_df.Longitude):
        stadium_coords = (float(i), float(j))
        distance_list.append(geopy.distance.distance(county_coords, stadium_coords).mi)
    max_distance.append(max(distance_list))

table_df['max_distance'] = max_distance

In [24]:
min_distance = None
closest_team = []
n = 0
for i, j in zip(table_df.Lat, table_df.Long):
    county_coords = (float(i), float(j))
    distance_list = []
    team_list = []
    for i, j, k in zip(stadiums_df.Latitude, stadiums_df.Longitude, stadiums_df.Team):
        stadium_coords = (float(i), float(j))
        distance_list.append(geopy.distance.distance(county_coords, stadium_coords).mi)
        team_list.append(k)
    min_distance = min(distance_list)
    min_index = distance_list.index(min_distance)
    if min_distance == table_df.at[n, 'min_distance']:
        closest_team.append(team_list[min_index])
    n += 1

table_df['closest_team'] = closest_team

max_distance = None
furthest_team = []
n = 0
for i, j in zip(table_df.Lat, table_df.Long):
    county_coords = (float(i), float(j))
    distance_list = []
    team_list = []
    for i, j, k in zip(stadiums_df.Latitude, stadiums_df.Longitude, stadiums_df.Team):
        stadium_coords = (float(i), float(j))
        distance_list.append(geopy.distance.distance(county_coords, stadium_coords).mi)
        team_list.append(k)
    max_distance = max(distance_list)
    max_index = distance_list.index(max_distance)
    if max_distance == table_df.at[n, 'max_distance']:
        furthest_team.append(team_list[max_index])
    n += 1

table_df['furthest_team'] = furthest_team

In [25]:
table_df.to_excel('county_latlong.xlsx')

In [26]:
table_df

,State,Geoid,County,Lat,Long,min_distance,closest_team,max_distance,furthest_team
0,AL,1001,Autauga,32.536382,-86.644490,156.853115,Atlanta Braves,2131.836047,Seattle Mariners
1,AL,1003,Baldwin,30.659218,-87.746067,293.899028,Atlanta Braves,2167.933281,Seattle Mariners
2,AL,1005,Barbour,31.870670,-85.405456,149.493454,Atlanta Braves,2216.944467,Seattle Mariners
3,AL,1007,Bibb,33.015893,-87.127148,165.009174,Atlanta Braves,2089.138163,Seattle Mariners
4,AL,1009,Blount,33.977448,-86.567246,120.732690,Atlanta Braves,2073.156985,Seattle Mariners
...,...,...,...,...,...,...,...,...,...
3138,WY,56037,Sweetwater,41.660339,-108.875676,242.458408,Colorado Rockies,1965.510481,Miami Marlins
3139,WY,56039,Teton,44.049321,-110.588102,413.302272,Colorado Rockies,2115.257562,Miami Marlins
3140,WY,56041,Uinta,41.284726,-110.558947,311.353699,Colorado Rockies,2035.360872,Miami Marlins
3141,WY,56043,Washakie,43.878831,-107.669052,316.227909,Colorado Rockies,1981.399586,Miami Marlins
